In [1]:
import sklearn, transformers, torch, tqdm, stanza, jieba
import numpy as np
import pandas as pd

/home/chwu/projects/gptdev/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('grave1.txt', 'r', encoding = "utf-16") as file:
    grave1 = file.read()
with open('grave2.txt', 'r', encoding = "utf-8") as file:
    grave2 = file.read()
with open('grave3.txt', 'r', encoding = "utf-8") as file:
    grave3 = file.read()
with open('grave4.txt', 'r', encoding = "utf-8") as file:
    grave4 = file.read()
with open('grave5.txt', 'r', encoding = "utf-8") as file:
    grave5 = file.read()
with open('grave6.txt', 'r', encoding = "utf-8") as file:
    grave6 = file.read()
with open('grave7.txt', 'r', encoding = "utf-8") as file:
    grave7 = file.read()
with open('grave8.txt', 'r', encoding = "utf-8") as file:
    grave8 = file.read()
with open('grave9.txt', 'r', encoding = "utf-8") as file:
    grave9 = file.read()
    
text = grave1+grave2+grave3+grave4+grave5+grave6+grave7+grave8+grave9

In [3]:
import stanza

nlp = stanza.Pipeline('zh', use_gpu=True, processors='tokenize')
def create_stanza_dict(text):
    doc = nlp(text)
    tokens = [word.text for sentence in doc.sentences for word in sentence.words]
    # Create a dictionary with each token and its corresponding index based on first appearance
    token_index_dict = {}
    current_order = 0
    for token in tokens:
        if token not in token_index_dict:
            token_index_dict[token] = current_order
            current_order += 1
    return token_index_dict

stanza_dict = create_stanza_dict(text)
print(list(stanza_dict.items())[:10])
print(list(stanza_dict.items())[-10:])
print(len(stanza_dict))


2024-06-08 13:41:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-08 13:41:23 INFO: Downloaded file to /home/chwu/stanza_resources/resources.json
2024-06-08 13:41:23 INFO: "zh" is an alias for "zh-hans"
2024-06-08 13:41:23 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |

/home/chwu/projects/gptdev/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at

[('～～～', 0), ('～', 1), ('小说', 2), ('下载尽', 3), ('在', 4), ('http://www.txt99.com——久久小说下载网附：【本作品来自互联网，本人不做任何负责】内容版权归作者所有!～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～', 5), ('《', 6), ('七', 7), ('星', 8), ('鲁', 9)]
[('误食', 42406), ('生来', 42407), ('肉饵', 42408), ('同日', 42409), ('似无', 42410), ('骗者', 42411), ('堂到', 42412), ('心腹', 42413), ('加以', 42414), ('【全书', 42415)]
42416


In [4]:
stanza_stoi = stanza_dict
stanza_itos = {index: char for char, index in stanza_dict.items()}
encode_stanza = lambda s: [stanza_stoi[c] for c in s if c in stanza_stoi]
decode_stanza = lambda l: ''.join([stanza_itos[i] for i in l if i in stanza_itos])

In [5]:
import torch
stanza_data = torch.tensor(encode_stanza(text), dtype=torch.long)
train_data = stanza_data[:int(0.9*len(stanza_data))]
val_data = stanza_data[int(0.9*len(stanza_data)):]

In [37]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 4000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 8
n_layer = 4
dropout = 0.2
vocab_size = len(stanza_dict)

In [7]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

11.700912 M parameters
step 0: train loss 10.8126, val loss 10.8140
step 100: train loss 5.8205, val loss 5.8452
step 200: train loss 5.2307, val loss 5.2451
step 300: train loss 4.9555, val loss 4.9580
step 400: train loss 4.7830, val loss 4.8134
step 500: train loss 4.6856, val loss 4.7165
step 600: train loss 4.6061, val loss 4.6473
step 700: train loss 4.5290, val loss 4.5898
step 800: train loss 4.4708, val loss 4.5271
step 900: train loss 4.4049, val loss 4.4911
step 1000: train loss 4.3805, val loss 4.4533
step 1100: train loss 4.3411, val loss 4.4254
step 1200: train loss 4.3077, val loss 4.4001
step 1300: train loss 4.2908, val loss 4.3503
step 1400: train loss 4.2580, val loss 4.3406
step 1500: train loss 4.2198, val loss 4.3283
step 1600: train loss 4.2096, val loss 4.2918
step 1700: train loss 4.1814, val loss 4.2827
step 1800: train loss 4.1613, val loss 4.2611
step 1900: train loss 4.1376, val loss 4.2534
step 2000: train loss 4.1179, val loss 4.2469
step 2100: train loss

In [40]:
print(decode_stanza(m.generate(context, max_new_tokens=300)[0].tolist()))

～～～是识的，潘子许我用手。”老痒里问：“我们看他们身光象没么样子，已经被放起了”那片儿，心说骂了几乎要往下换了之前，我一下我想到那只要衣服露出水枪。胖子倒那老。几乎高加索我一楞是以前一般，然后打了她就烧酒死了，嘴巴，我们全飞了下来，里面上对我们半略道：“你们得和阿宁怎么回事儿，咱们只能拍了！”他指了指胖子放下给看他们？如果和你们高采的挖了，毕外面他两句话算着玩，所以她能容易都越没有话，差他让做的事情和他说大，你的是是么老人。”凉师这事情也不是自己京矿，那些墓室的乌大，每灵这上一群，我又不说，千万一场几种马大往队的时就被人把我绑住起潘子，脸上全部的画上装备，全部跟营，看到那些方面肝迫起了树后透探灯的


In [30]:
test1 = '冥殿四面牆壁倒不是什麽都没有，只有些打底的壁画，都是白描，还没有进行上色，画的是日月星辰。'

test2 = "该局高度重视，立即组织执法人员对该事件进行调查，对当事摊主进行了问询，摊主已承认其存在缺斤少两行为。"

test3 = "面自童奶里且平由干笑五浪，禾尺要见姐语绿吗民隻实蝶各、反下反第几她次视亭。"

test4 = "崩贴喘灵存匈铸同刃啬曾贝垮邓？敷仰吁左骗盘备。插护，炭泌倒，臭酿层述？"

encodings = torch.tensor(encode_stanza(test1), dtype=torch.long).unsqueeze(0).to(device)
input_ids = encodings[:, 0:block_size].to(device)
target_ids = input_ids.clone()
with torch.no_grad():
  outputs, loss = m(input_ids, targets=target_ids)
outputs = outputs.unsqueeze(0)
probs = F.softmax(outputs, dim=-1)
torch.manual_seed(1337)
import torchmetrics
from torchmetrics.text import Perplexity

perp = Perplexity(ignore_index=-100).to(device)
perp(outputs, target_ids)


tensor(962.5798)

In [34]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

encoding.encode("该局高度重视")

[76982, 91077, 45736, 27479, 30358, 58552]

In [35]:
encoding.decode([76982])

'该'